# NetCDF zu GeoTIFF Konvertierung
Dieses Skript lädt NetCDF-Dateien, extrahiert relevante Daten, und speichert sie als GeoTIFF-Dateien. Da NetCDF-Dateien oft Zeitschritte enthalten, wird das Skript die Daten über die Zeitachsen iterieren und mehrere TIFF-Dateien erzeugen (eines pro Zeitschritt).

### Bibliotheken:
- `xarray`: Für das Laden und Bearbeiten von NetCDF-Daten.
- `rasterio`: Für die Erstellung von GeoTIFF-Dateien.
- `numpy`: Für die Arbeit mit numerischen Daten.

### Vorgehen:
1. Bibliotheken instllieren, laden und NetCDF-Datei öffnen.
2. Überprüfen der Datenstruktur (Dimensionen, Variablen).
3. Konvertierung der NetCDF-Daten (pro Zeitschritt) in GeoTIFF.

# Verarbeitung von NetCDF-Dateien
In diesem Notebook werden NetCDF-Dateien verarbeitet, um Niederschlagsdaten zu extrahieren und in das GeoTIFF-Format zu konvertieren. Der Benutzer kann entweder spezifische Zeitschritte angeben oder alle Zeitschritte auf einmal verarbeiten. 
Die Ergebnisse werden in einem festgelegten Ausgabeverzeichnis gespeichert, und eine Logdatei dokumentiert die Struktur der ersten Datei.

In [1]:
# Funktion zur Prüfung und Installation von Modulen
import sys
import subprocess

def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} ist nicht installiert. Installation wird durchgeführt...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        globals()[package] = __import__(package)

# Liste der benötigten Module
modules = ['xarray', 'rasterio', 'numpy', 'os', 'packaging', 'pandas', 'netCDF4', 'h5netcdf', 'h5py', 'scipy']

# Prüfen und ggf. Installieren der Module
for module in modules:
    install_and_import(module)


In [2]:
# Bibliotheken importieren
import xarray as xr
import rasterio
import numpy as np
import os
import packaging
import pandas
import netCDF4
import h5netcdf
import h5py
import scipy


## Schritt 1: NetCDF-Datei laden
Verwende die Bibliothek `xarray`, um die NetCDF-Datei zu öffnen und einen Überblick über die enthaltenen Daten und Dimensionen zu bekommen.
## Ordnerpfade festlegen

Wir definieren den Pfad zum Eingangsordner, der die NetCDF-Dateien enthält, sowie den Pfad zum Ausgangsordner, in dem die GeoTIFF-Dateien und das Logfile gespeichert werden.

In [32]:
# Ordnerpfad für die Eingabedaten und Ausgabedaten
input_folder_path = './data/01/'  # Beispielpfad anpassen
output_folder_path = './output/'  # Beispielpfad anpassen


## NetCDF-Dateien auflisten

Hier definieren wir eine Funktion, die alle NetCDF-Dateien im angegebenen Eingangsordner auflistet. Diese Funktion wird aufgerufen, um die zu verarbeitenden Dateien zu sammeln.


In [33]:
# Funktion zur Auflistung aller .nc Dateien im Ordner
def get_nc_files_from_folder(folder_path):
    nc_files = []
    for root, _, files in os.walk(folder_path):  # Durchlaufe alle Unterordner
        for file in files:
            if file.endswith('.nc'):  # Prüfe auf .nc Dateien
                nc_files.append(os.path.join(root, file))  # Füge den vollständigen Pfad zur Liste hinzu
    return nc_files

# Alle .nc Dateien im Ordner finden
nc_files = get_nc_files_from_folder(input_folder_path)

# Zeige alle gefundenen .nc Dateien an
print(f"Gefundene .nc Dateien: {nc_files}")


Gefundene .nc Dateien: ['./data/01/YW_2017.002_20210101.nc', './data/01/YW_2017.002_20210102.nc', './data/01/YW_2017.002_20210103.nc', './data/01/YW_2017.002_20210104.nc', './data/01/YW_2017.002_20210105.nc', './data/01/YW_2017.002_20210106.nc', './data/01/YW_2017.002_20210107.nc', './data/01/YW_2017.002_20210108.nc', './data/01/YW_2017.002_20210109.nc', './data/01/YW_2017.002_20210110.nc', './data/01/YW_2017.002_20210111.nc', './data/01/YW_2017.002_20210112.nc', './data/01/YW_2017.002_20210113.nc', './data/01/YW_2017.002_20210114.nc', './data/01/YW_2017.002_20210115.nc', './data/01/YW_2017.002_20210116.nc', './data/01/YW_2017.002_20210117.nc', './data/01/YW_2017.002_20210118.nc', './data/01/YW_2017.002_20210119.nc', './data/01/YW_2017.002_20210120.nc', './data/01/YW_2017.002_20210121.nc', './data/01/YW_2017.002_20210122.nc', './data/01/YW_2017.002_20210123.nc', './data/01/YW_2017.002_20210124.nc', './data/01/YW_2017.002_20210125.nc', './data/01/YW_2017.002_20210126.nc', './data/01/YW_

## Logfile erstellen

Hier erstellen wir ein Logfile, das die Struktur und die Attribute des ersten Datensatzes protokolliert, der verarbeitet wird.


In [34]:
# Logfile erstellen
log_file_path = os.path.join(output_folder_path, 'log.txt')

# Funktion zur Protokollierung der Datenstruktur und Attribute
def log_dataset_structure(ds, log_file):
    with open(log_file, 'a') as f:
        f.write(f"Dataset Struktur:\n")
        f.write(str(ds) + "\n\n")  # Schreibe die Struktur des Datensatzes
        f.write(f"Attribute:\n")
        for attr in ds.attrs:
            f.write(f"{attr}: {ds.attrs[attr]}\n")  # Schreibe die Attribute des Datensatzes
        f.write("\n" + "="*40 + "\n\n")

# Protokolliere die Struktur und Attribute der ersten Datei
def log_first_dataset_structure(nc_file, log_file):
    ds = xr.open_dataset(nc_file, engine='netcdf4')  # Lade den Datensatz
    log_dataset_structure(ds, log_file)  # Logge die Struktur
    return ds


## Zeitschrittauswahl und Prozessierung

Hier definieren wir eine Funktion, die es dem Benutzer ermöglicht, die gewünschten Zeitschritte auszuwählen, um die Niederschlagsdaten zu extrahieren.


In [35]:
# Funktion zur Umwandlung der Eingabe in eine Liste von Zeitschritten
def parse_time_steps(input_str, ds):
    if input_str.strip().lower() == "0-1":  # Wenn 'all' eingegeben wird
        return list(range(len(ds['time'])))  # Gib alle Zeitschritte zurück
    else:
        time_steps = []
        ranges = input_str.split(",")  # Splitte die Eingabe nach Kommas
        for r in ranges:
            if "-" in r:  # Prüfe auf einen Bereich
                start, end = map(int, r.split("-"))  # Splitte den Bereich
                time_steps.extend(range(start, end + 1))  # Füge den Bereich zur Liste hinzu
            else:
                time_steps.append(int(r))  # Füge einzelne Zeitstempel zur Liste hinzu
        return time_steps

# Funktion zur Extraktion der Zeitschritte und Datenvariablen
def process_time_steps(nc_file, time_step_input):
    ds = xr.open_dataset(nc_file, engine='netcdf4')  # Lade die aktuelle Datei
    selected_time_steps = parse_time_steps(time_step_input, ds)  # Bestimme die zu exportierenden Zeitschritte
    
    # Überprüfe die CRS-Variable und greife auf ihre proj4-Attribute zu
    if 'crs' in ds.variables:
        crs_value = ds['crs'].attrs.get('proj4', 'Keine gültige CRS-Information im proj4-Format vorhanden')
        print(f"CRS-Proj4-String: {crs_value}")
    else:
        crs_value = None
        print("Keine CRS-Variable gefunden.")
    
    return ds, selected_time_steps, crs_value


## GeoTIFF-Export

Hier definieren wir eine Funktion, die die Niederschlagsdaten für jeden ausgewählten Zeitschritt in eine GeoTIFF-Datei exportiert. Außerdem bereinigen wir die Dateinamen, um Probleme mit Punkten zu vermeiden.


In [36]:
# Funktion zur Bereinigung des Dateinamens
def clean_filename(filename):
    cleaned_name = re.sub(r'\.', '_', filename[:filename.rfind('.')]) + filename[filename.rfind('.'):]  # Ersetze Punkte durch Unterstriche
    return cleaned_name

# Metadaten (Transform) für die GeoTIFF-Erstellung
def get_transform(ds):
    return from_origin(min(ds['lon'].values.flatten()), max(ds['lat'].values.flatten()), abs(ds['x'][1] - ds['x'][0]), abs(ds['y'][1] - ds['y'][0]))  # Berechne den Transformationsparameter

# Funktion zur Konvertierung eines Zeitschritts in ein GeoTIFF
def export_to_geotiff(time_index, ds, crs_value, transform, nc_file_path):
    rr_data = ds['RR'].isel(time=time_index)  # Niederschlagsdaten für den Zeitschritt
    timestamp = pd.to_datetime(str(ds['time'].values[time_index]))  # Zeitstempel
    
    # Dateinamen generieren
    nc_filename = os.path.basename(nc_file_path).split('.')[0]  # Basisdateiname extrahieren
    cleaned_nc_filename = clean_filename(nc_filename)  # Bereinige den Dateinamen
    variable_name = 'RR'  # Variable, die exportiert wird
    output_filename = f"{cleaned_nc_filename}_{variable_name}_{timestamp.strftime('%Y%m%d_%H%M%S')}.tif"  # Erstelle den Ausgabedateinamen
    
    # Output-Pfad erstellen
    output_tiff_path = os.path.join(output_folder_path, output_filename)
    
    # Schreibe die Daten in eine GeoTIFF-Datei
    with rasterio.open(
        output_tiff_path, 'w',
        driver='GTiff',
        height=rr_data.shape[0],
        width=rr_data.shape[1],
        count=1,
        dtype=rr_data.dtype,
        crs=crs_value,  # Verwende die CRS-Information aus der Datei
        transform=transform
    ) as dst:
        dst.write(rr_data.values, 1)  # Schreibe die Niederschlagsdaten in die GeoTIFF-Datei
    
    print(f"GeoTIFF für Zeitschritt {time_index} mit Zeitstempel {timestamp} wurde erfolgreich unter {output_tiff_path} gespeichert.")


## Verarbeitung aller Dateien

Hier definieren wir eine Funktion, die alle .nc-Dateien im angegebenen Ordner verarbeitet. Für die erste Datei werden die Struktur und die Attribute protokolliert.


In [37]:
# Funktion zur Verarbeitung aller .nc Dateien im Ordner
def process_all_nc_files(nc_files, time_step_input, log_file):
    first_ds_logged = False  # Flag, um zu prüfen, ob die Struktur der ersten Datei geloggt wurde
    for nc_file in nc_files:
        print(f"Verarbeite Datei: {nc_file}")
        
        # Logge die Struktur des ersten Datensatzes
        if not first_ds_logged:
            ds = log_first_dataset_structure(nc_file, log_file)  # Protokolliere die Struktur der ersten Datei
            first_ds_logged = True
        else:
            ds = xr.open_dataset(nc_file, engine='netcdf4')  # Lade die aktuelle Datei
            
        selected_time_steps = parse_time_steps(time_step_input, ds)  # Bestimme die zu exportierenden Zeitschritte
        crs_value = ds['crs'].attrs.get('proj4', 'Keine gültige CRS-Information im proj4-Format vorhanden')  # Überprüfe CRS
        transform = get_transform(ds)  # Berechne den Transformationsparameter
        
        # Exportiere die Daten für jeden ausgewählten Zeitschritt
        for time_index in selected_time_steps:
            export_to_geotiff(time_index, ds, crs_value, transform, nc_file)  # Exportiere die GeoTIFF-Datei

# Verarbeite alle Dateien im angegebenen Ordner
process_all_nc_files(nc_files, time_step_input, log_file_path)  # Starte die Verarbeitung


Verarbeite Datei: ./data/01/YW_2017.002_20210101.nc
GeoTIFF für Zeitschritt 0 mit Zeitstempel 2021-01-01 00:00:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_000000.tif gespeichert.
GeoTIFF für Zeitschritt 1 mit Zeitstempel 2021-01-01 00:05:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_000500.tif gespeichert.
GeoTIFF für Zeitschritt 2 mit Zeitstempel 2021-01-01 00:10:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_001000.tif gespeichert.
GeoTIFF für Zeitschritt 3 mit Zeitstempel 2021-01-01 00:15:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_001500.tif gespeichert.
Verarbeite Datei: ./data/01/YW_2017.002_20210102.nc
GeoTIFF für Zeitschritt 0 mit Zeitstempel 2021-01-02 00:00:00 wurde erfolgreich unter ./output/YW_2017_RR_20210102_000000.tif gespeichert.
GeoTIFF für Zeitschritt 1 mit Zeitstempel 2021-01-02 00:05:00 wurde erfolgreich unter ./output/YW_2017_RR_20210102_000500.tif gespeichert.
GeoTIFF für Zeitschritt 2 mit Zeitstempel 2021-01-02 00:10:00 

# =================================================================

In [7]:
# Extrahiere die 'crs'-Information und zeige sie an
if 'crs' in ds.variables:
    crs_value = ds['crs'].attrs.get('proj4', 'Keine gültige CRS-Information im proj4-Format vorhanden')
    print(f"CRS-Proj4-String: {crs_value}")
else:
    crs_value = None
    print("Keine CRS-Variable gefunden.")

CRS-Proj4-String: +proj=stere +lat_0=90 +lat_ts=90 +lon_0=10 +k_0=0.933012701892 +x_0=0 +y_0=0 +a=6370040 +b=6370040 +units=km +no_defs


## Schritt 2: Auswahl der Variable und des Zeitintervalls
Wähle die Variable aus der NetCDF-Datei, die du konvertieren möchtest (z.B. Temperatur, Niederschlag, etc.), und definiere den Zeitschritt, den du in eine GeoTIFF-Datei umwandeln möchtest.


In [18]:
import re

# Zeitschrittauswahl (manuelle Eingabe)
# Standard ist "all", aber kann auf spezifische Zeitschritte oder Bereiche angepasst werden.
time_step_input = "all"  # Beispiel: "0-10, 15, 20-25"
time_step_input = "0-3"

# Funktion zur Umwandlung der Eingabe in eine Liste von Zeitschritten
def parse_time_steps(input_str, ds):
    if input_str.strip().lower() == "all":
        return list(range(len(ds['time'])))
    else:
        time_steps = []
        ranges = input_str.split(",")
        for r in ranges:
            if "-" in r:
                start, end = map(int, r.split("-"))
                time_steps.extend(range(start, end + 1))
            else:
                time_steps.append(int(r))
        return time_steps

# Verarbeite die Eingabe der Zeitschritte
selected_time_steps = parse_time_steps(time_step_input, ds)

# Falls keine gültige Auswahl getroffen wurde
if not selected_time_steps:
    print("Keine gültige Auswahl getroffen. Vorgang wird abgebrochen.")
else:
    print(f"Ausgewählte Zeitschritte: {selected_time_steps}")


Ausgewählte Zeitschritte: [0, 1, 2, 3]


In [24]:
import rasterio
from rasterio.transform import from_origin
import pandas as pd
import re

# Funktion zur Bereinigung des Dateinamens
def clean_filename(filename):
    # Ersetze Punkte (.) im Dateinamen durch Unterstriche, außer der letzten für die Dateiendung
    cleaned_name = re.sub(r'\.', '_', filename[:filename.rfind('.')]) + filename[filename.rfind('.'):]
    return cleaned_name

# Metadaten (Transform) für die GeoTIFF-Erstellung
transform = from_origin(min(ds['lon'].values.flatten()), max(ds['lat'].values.flatten()), abs(ds['x'][1] - ds['x'][0]), abs(ds['y'][1] - ds['y'][0]))

# Funktion zur Konvertierung eines Zeitschritts in ein GeoTIFF
def export_to_geotiff(time_index, ds, crs_value, transform, nc_file_path):
    rr_data = ds['RR'].isel(time=time_index)  # Niederschlagsdaten für den Zeitschritt
    timestamp = pd.to_datetime(str(ds['time'].values[time_index]))  # Zeitstempel
    
    # Dateinamen generieren
    nc_filename = os.path.basename(nc_file_path).split('.')[0]
    cleaned_nc_filename = clean_filename(nc_filename)
    variable_name = 'RR'  # Variable, die exportiert wird
    output_filename = f"{cleaned_nc_filename}_{variable_name}_{timestamp.strftime('%Y%m%d_%H%M%S')}.tif"
    
    output_tiff = f'./output/{output_filename}'
    
    # Schreibe die Daten in eine GeoTIFF-Datei
    with rasterio.open(
        output_tiff, 'w',
        driver='GTiff',
        height=rr_data.shape[0],
        width=rr_data.shape[1],
        count=1,
        dtype=rr_data.dtype,
        crs=crs_value,  # Verwende die CRS-Information aus der Datei
        transform=transform
    ) as dst:
        dst.write(rr_data.values, 1)
    
    print(f"GeoTIFF für Zeitschritt {time_index} mit Zeitstempel {timestamp} wurde erfolgreich unter {output_tiff} gespeichert.")

# Iteriere über die ausgewählten Zeitschritte und exportiere sie als GeoTIFF
for time_index in selected_time_steps:
    export_to_geotiff(time_index, ds, crs_value, transform, nc_file_path)


GeoTIFF für Zeitschritt 0 mit Zeitstempel 2021-01-01 00:00:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_000000.tif gespeichert.
GeoTIFF für Zeitschritt 1 mit Zeitstempel 2021-01-01 00:05:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_000500.tif gespeichert.
GeoTIFF für Zeitschritt 2 mit Zeitstempel 2021-01-01 00:10:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_001000.tif gespeichert.
GeoTIFF für Zeitschritt 3 mit Zeitstempel 2021-01-01 00:15:00 wurde erfolgreich unter ./output/YW_2017_RR_20210101_001500.tif gespeichert.


## Schritt 3: Konvertierung in GeoTIFF
Nun konvertieren wir die extrahierten Rasterdaten in eine GeoTIFF-Datei. Dafür wird die Bibliothek `rasterio` verwendet. Wir müssen auch die Geoinformationen wie die Projektion (z.B. WGS84) und die Bounding Box bereitstellen.


In [ ]:
# Erstellen eines GeoTIFF-Dateipfads
output_tif = 'output_raster.tif'

# Beispielhafte affine Transformation und CRS (proj4 oder EPSG-Code)
transform = rasterio.transform.from_origin(west=0, north=0, xsize=0.1, ysize=0.1)  # Beispielhafte Parameter
crs = 'EPSG:4326'  # WGS84

# Öffnen einer neuen TIFF-Datei zum Schreiben
with rasterio.open(
    output_tif, 
    'w', 
    driver='GTiff', 
    height=data_at_time.shape[0], 
    width=data_at_time.shape[1], 
    count=1, 
    dtype=str(data_at_time.dtype), 
    crs=crs, 
    transform=transform
) as dst:
    # Schreibe die Daten in die TIFF-Datei
    dst.write(data_at_time.values, 1)

print(f'Datei {output_tif} erfolgreich erstellt.')


## Schritt 4: Konvertierung für alle Zeitschritte (optional)
Falls die NetCDF-Datei mehrere Zeitschritte enthält und du alle in separate TIFF-Dateien konvertieren möchtest, kannst du über die Zeitachse iterieren und für jeden Zeitschritt eine TIFF-Datei erzeugen.


In [ ]:
# Erstellen eines Ordners zum Speichern der GeoTIFFs
output_folder = 'output_tiffs'
os.makedirs(output_folder, exist_ok=True)

# Über die Zeitachsen iterieren
for i in range(len(ds.coords['time'])):
    # Extrahiere Daten für den aktuellen Zeitschritt
    data_at_time = data_var.isel(time=i)
    
    # Dateiname für den aktuellen Zeitschritt
    output_tif = os.path.join(output_folder, f'raster_time_{i}.tif')
    
    # GeoTIFF-Datei erstellen
    with rasterio.open(
        output_tif, 
        'w', 
        driver='GTiff', 
        height=data_at_time.shape[0], 
        width=data_at_time.shape[1], 
        count=1, 
        dtype=str(data_at_time.dtype), 
        crs=crs, 
        transform=transform
    ) as dst:
        dst.write(data_at_time.values, 1)

    print(f'Datei {output_tif} für Zeitschritt {i} erfolgreich erstellt.')


## Fazit
Das Skript ermöglicht die Konvertierung von NetCDF-Daten in GeoTIFF-Dateien, wobei Zeitschritte berücksichtigt werden. Diese Methode kann verwendet werden, um rasterbasierte Zeitreihenanalysen in einem georeferenzierten Format (TIFF) zu speichern und weiterzuverarbeiten.
